Following: https://openpolicedata.readthedocs.io/en/stable/getting_started/index.html

In [2]:
# %pip install openpolicedata

In [2]:
import openpolicedata as opd
import pandas as pd

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# table_types = opd.datasets.get_table_types(contains="STOPS")
# print(table_types)

['PEDESTRIAN STOPS', 'STOPS', 'STOPS - INCIDENTS', 'STOPS - SUBJECTS', 'TRAFFIC STOPS', 'TRAFFIC STOPS - INCIDENTS', 'TRAFFIC STOPS - SUBJECTS']


In [ ]:
# # Load traffic stops data from Lousiville for the year 2022.
# src = opd.Source("Louisville")
# tbl = src.load(table_type="TRAFFIC STOPS" ,year=2022)

In [ ]:
# View the 1st 5 rows with pandas' head function
# tbl.table.head()

Finding Datasets
OPD provides the datasets module for querying what datasets are available in OPD. To get all available datasets, query the source table with no inputs:

In [3]:
all_datasets = opd.datasets.query()

In [4]:
all_datasets.filter(items=['source_url']).head(10)

,source_url
0,https://data.chandlerpd.com/catalog/arrest-boo...
1,https://data.chandlerpd.com/catalog/calls-for-...
2,https://data.chandlerpd.com/catalog/general-of...
3,https://data.gilbertaz.gov/maps/2dcb4c20c9a444...
4,https://data.gilbertaz.gov/datasets/TOG::gilbe...
5,https://openpolicing.stanford.edu/data/
6,https://data.mesaaz.gov/Police/Police-Computer...
7,https://data.mesaaz.gov/Police/Police-Dispatch...
8,https://data.mesaaz.gov/Police/Police-Incident...
9,https://openpolicing.stanford.edu/data/


Script needs to:
1. Check "Last Checked Date" (by surragate key for police_data_source_tracking.csv)
    - Determine "outdated" sources to try depending on dates. i.e. if date is older than a day, month or year
2. If Last Checked is outdated, fetch max coverage end then corresponding source_url
3. find year in url
4. replace year in url with coverage_end year + 1
5. try to open new url and return data found true or false
    - Test may need to vary based on DataType

In [10]:
#get all unique values of datatype
all_datasets['DataType'].unique()

array(['CSV', 'ArcGIS', 'Socrata', 'CKAN', 'Excel', 'HTML', 'Carto'],
      dtype=object)

# Look for Predicable URLS

In [ ]:
# Define the columns to match for predictability
columns_to_match = ['State', 'SourceName', 'AgencyFull', 'TableType']

# Group by the specified columns and filter groups where the URL is predictable
def is_url_predictable(group):
    # Check if all URLs in the group are identical
    return group['URL'].nunique() == 1

# Apply the filter
predictable_urls = all_datasets.groupby(columns_to_match).filter(is_url_predictable)

# Display the filtered dataset
predictable_urls

# Look for URLs worth checking

##### URLs with only one set of 4 sequential digits?

In [11]:
# Filter all_datasets by URL containing 4 digits starting with 19 or 20
filtered_records = all_datasets[all_datasets['URL'].str.contains(r'\b(19|20)\d{2}\b')]
#filtered_records = filtered_records.groupby(filtered_records['URL'].str.replace(r'\d{4}', '####', regex=True)).filter(lambda x: len(x) > 1)

filtered_records[['State', 'SourceName', 'coverage_start', 'coverage_end', 'Year', 'last_coverage_check', 'AgencyFull', 'TableType', 'source_url', 'URL']].tail(15)

C:\Users\Ashley\AppData\Local\Temp\ipykernel_10716\3813301252.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered_records = all_datasets[all_datasets['URL'].str.contains(r'\b(19|20)\d{2}\b')]


,State,SourceName,coverage_start,coverage_end,Year,last_coverage_check,AgencyFull,TableType,source_url,URL
1433,Vermont,Winooski,2014-01-01,2014-12-31,2014,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/785/2014-raw-traffic-stop-and-ticket-data-spread-sheet
1434,Vermont,Winooski,2015-01-01,2015-12-31,2015,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/786/2015-wpd-raw-traffic-stop-and-ticket-data-spread-sheet
1435,Vermont,Winooski,2016-01-01,2016-12-31,2016,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/787/2016-raw-traffic-stop-and-ticket-data-spread-sheet
1436,Vermont,Winooski,2017-01-01,2017-12-31,2017,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/788/2017-raw-traffic-stop-and-ticket-data-spread-sheet1
1437,Vermont,Winooski,2018-01-01,2018-12-31,2018,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/780/2018-raw-traffic-stop-and-ticket-data-spread-sheet
1438,Vermont,Winooski,2019-01-01,2019-12-31,2019,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/2886/2019--Raw-Traffic-Stop-and-Ticket-Data
1439,Vermont,Winooski,2020-01-01,2020-12-31,2020,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/4301/2020-Raw-Traffic-Stop-Data
1440,Vermont,Winooski,2021-01-01,2021-12-31,2021,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/5560/2021-Raw-Traffic-Stop-Data
1441,Vermont,Winooski,2022-01-01,2022-12-31,2022,07/07/2023,Winooski Police Department,TRAFFIC STOPS,https://www.winooskivt.gov/355/Fair-and-Impartial-Policing-Reporting,https://www.winooskivt.gov/DocumentCenter/View/6638/2022-Raw-Traffic-Stop-Data
1512,Washington,Yakima County,2016-01-01,2016-12-31,2016,12/27/2023,Yakima County Sheriff's Office,USE OF FORCE,https://www.muckrock.com/foi/yakima-county-8575/sheriff-data-collection-request-2016-2019-149772/#comms,https://cdn.muckrock.com/foia_files/2023/08/15/2016_Use_of_Force_Log.xlsx


##### Edge Case

- URLs are predictable, just need to try changing more than the year in some cases, i.e. "https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-12/RIPA-Stop-Data-2023.zip"
- https://www.winooskivt.gov/DocumentCenter/View/780/2018-raw-traffic-stop-and-ticket-data-spread-sheet is downloading when changing to https://www.winooskivt.gov/DocumentCenter/View/780/2019-raw-traffic-stop-and-ticket-data-spread-sheet

### Ideas

- Add unique key to the above table
- Add column for how the source was received, i.e. public record, public records request, etc.
- normalize OPD datasets
    - generate unique keys based on surrogate, last coverage check can apply to key

In [21]:
filtered_records.head(5)

,State,SourceName,Agency,AgencyFull,TableType,coverage_start,coverage_end,last_coverage_check,Year,agency_originated,supplying_entity,Description,source_url,readme,URL,DataType,date_field,dataset_id,agency_field,min_version,py_min_version,query
75,California,Alameda County,MULTIPLE,NaN,STOPS,2021-01-01,2021-12-31,09/13/2024,2021,yes,California Office of the Attorney General,Assembly Bill 953 (AB 953) requires each state and local agency in California that employs peace officers to annually report to the Attorney General data on all stops conducted by the agency's peace officers,https://openjustice.doj.ca.gov/data,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2023-03/RIPA%20Dataset%20Read%20Me%202021%20Final.pdf,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2023-03/RIPA-Stop-Data-2021.zip,Excel,<NA>,RIPA Stop Data 2021/RIPA Stop Data _ Alameda 2021.xlsx,AGENCY_NAME,0.8.2,<NA>,NaN
76,California,Alameda County,MULTIPLE,NaN,STOPS,2022-01-01,2022-12-31,09/13/2024,2022,yes,California Office of the Attorney General,Assembly Bill 953 (AB 953) requires each state and local agency in California that employs peace officers to annually report to the Attorney General data on all stops conducted by the agency's peace officers,https://openjustice.doj.ca.gov/data,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-01/RIPA%20Dataset%20Read%20Me%202022.pdf,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2023-12/RIPA-Stop-Data-2022.zip,Excel,<NA>,RIPA Stop Data _ Alameda 2022.xlsx,AGENCY_NAME,0.8.2,<NA>,NaN
77,California,Alameda County,MULTIPLE,NaN,STOPS,2023-01-01,2023-12-31,04/06/2025,2023,yes,California Office of the Attorney General,Assembly Bill 953 (AB 953) requires each state and local agency in California that employs peace officers to annually report to the Attorney General data on all stops conducted by the agency's peace officers,https://openjustice.doj.ca.gov/data,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-12/RIPA%20Dataset%20Read%20Me%202023%20Final.pdf,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-12/RIPA-Stop-Data-2023.zip,Excel,<NA>,Tables/RIPA Stop Data _ Alameda 2023 final.xlsx,AGENCY_NAME,0.8.2,<NA>,NaN
78,California,Alpine County,Alpine County,Alpine County Sheriff's Office,STOPS,2022-01-01,2022-12-31,09/13/2024,2022,yes,California Office of the Attorney General,Assembly Bill 953 (AB 953) requires each state and local agency in California that employs peace officers to annually report to the Attorney General data on all stops conducted by the agency's peace officers,https://openjustice.doj.ca.gov/data,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-01/RIPA%20Dataset%20Read%20Me%202022.pdf,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2023-12/RIPA-Stop-Data-2022.zip,Excel,<NA>,RIPA Stop Data _ Alpine 2022.xlsx,AGENCY_NAME,0.8.2,<NA>,NaN
79,California,Alpine County,Alpine County,Alpine County Sheriff's Office,STOPS,2023-01-01,2023-12-31,04/06/2025,2023,yes,California Office of the Attorney General,Assembly Bill 953 (AB 953) requires each state and local agency in California that employs peace officers to annually report to the Attorney General data on all stops conducted by the agency's peace officers,https://openjustice.doj.ca.gov/data,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-12/RIPA%20Dataset%20Read%20Me%202023%20Final.pdf,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-12/RIPA-Stop-Data-2023.zip,Excel,<NA>,Tables/RIPA Stop Data _ Alpine 2023 final.xlsx,AGENCY_NAME,0.8.2,<NA>,NaN


# Misc

In [ ]:
all_datasets[all_datasets['source_url'] == 'https://opendata.baltimorecity.gov/egis/rest/services/Hosted/911_Calls_For_Service_2017_csv/FeatureServer/0']

,State,SourceName,Agency,AgencyFull,TableType,coverage_start,coverage_end,last_coverage_check,Year,agency_originated,supplying_entity,Description,source_url,readme,URL,DataType,date_field,dataset_id,agency_field,min_version,py_min_version,query


In [ ]:
# Group by URLs with years replaced by '####' and filter for increasing years
def has_increasing_years(group):
    years = group['URL'].str.extract(r'(\b\d{4}\b)').dropna().astype(int).sort_values(by=0)
    return years[0].is_monotonic_increasing

filtered_records = all_datasets.groupby(all_datasets['URL'].str.replace(r'\b\d{4}\b', '####', regex=True)).filter(has_increasing_years)

TypeError: Index(...) must be called with a collection of some kind, <function has_increasing_years at 0x0000019B8840CB80> was passed

# Learn these
- row_number
- rank
- rank_dense

In [ ]:
filtered_records

,State,SourceName,Agency,AgencyFull,TableType,coverage_start,coverage_end,last_coverage_check,Year,agency_originated,supplying_entity,Description,source_url,readme,URL,DataType,date_field,dataset_id,agency_field,min_version,py_min_version,query
0,Arizona,Chandler,Chandler,Chandler Police Department,ARRESTS,2018-01-01,2025-02-16,02/17/2025,MULTIPLE,NaN,NaN,"Arrest reports completed by a Chandler Police Department officer. Arrest reports are made when one of the following occurs: An adult or juvenile suspect is taken into custody and booked into jail, An adult is arrested and taken into custody, but released prior to booking, pending the submittal of charge, A juvenile is referred to court for criminal charges, including status offenses such as runaway and curfew, A criminal citation is issued. Location data is generalized to the nearest block to protect victim privacy.",https://data.chandlerpd.com/catalog/arrest-bookings/,<NA>,https://data.chandlerpd.com/catalog/arrest-bookings/download/csv/,CSV,arrest_date_time,NaN,<NA>,0.2,<NA>,NaN
1,Arizona,Chandler,Chandler,Chandler Police Department,CALLS FOR SERVICE,2018-01-01,2025-02-17,02/17/2025,MULTIPLE,NaN,NaN,"This dataset contains details for all of the calls for service that have been reported to the Chandler Police Department. The calls for service can be generated in four primary ways: emergency 911 phone calls, non-emergency phone calls or text messages, officer self-initiated, and alarms. Location data is generalized to the nearest block to protect victim privacy.",https://data.chandlerpd.com/catalog/calls-for-service/,<NA>,https://data.chandlerpd.com/catalog/calls-for-service/download/csv/,CSV,call_received_date_time,NaN,<NA>,<NA>,<NA>,NaN
2,Arizona,Chandler,Chandler,Chandler Police Department,INCIDENTS,2018-01-01,2025-02-10,02/17/2025,MULTIPLE,NaN,NaN,This dataset contains details for all of the general offense reports completed by a Chandler Police Department officer. Location data is generalized to the nearest block to protect victim privacy.,https://data.chandlerpd.com/catalog/general-offenses/,<NA>,https://data.chandlerpd.com/catalog/general-offenses/download/csv/,CSV,report_event_date,NaN,<NA>,0.4.1,<NA>,NaN
3,Arizona,Gilbert,Gilbert,Gilbert Police Department,CALLS FOR SERVICE,2006-11-15,2025-02-16,02/17/2025,MULTIPLE,NaN,NaN,<NA>,https://data.gilbertaz.gov/maps/2dcb4c20c9a444f2a5d3dca180b651c9,<NA>,https://maps.gilbertaz.gov/arcgis/rest/services/OD/Community_Safety_Tables_1/MapServer/2,ArcGIS,EventDate,NaN,<NA>,<NA>,<NA>,NaN
4,Arizona,Gilbert,Gilbert,Gilbert Police Department,EMPLOYEE,NaT,NaT,07/06/2023,NONE,NaN,NaN,A data set of all employees that have previously worked for or currently work for the Town of Gilbert highlighting demographics.,https://data.gilbertaz.gov/datasets/TOG::gilbert-demographics/about,<NA>,https://services1.arcgis.com/JLuzSHjNrLL4Okwb/arcgis/rest/services/Gilbert_Demographics/FeatureServer/0,ArcGIS,<NA>,NaN,<NA>,<NA>,<NA>,{“Department”: “POLICE DEPARTMENT”}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1325,Wisconsin,Beloit,Beloit,Beloit Police Department,USE OF FORCE,2020-01-01,2020-12-31,07/07/2023,2020,NaN,NaN,<NA>,https://www.beloitwi.gov/index.asp?SEC=3209CC74-34C0-4358-A679-9B061EE965C3&DE=FA507A9C-2ED5-4E8C-B098-8F3E78568AF6,<NA>,http://gouda.beloitwi.gov/WebLink/0/edoc/72935/1Use%20of%20Force%202020%20-%20Updated%20on%206-12-2020.xls,Excel,<NA>,NaN,<NA>,0.3.1,<NA>,NaN
1326,Wisconsin,Madison,Madison,Madison Police Department,TRAFFIC STOPS,2007-09-28,2020-06-16,01/10/2024,MULTIPLE,yes,Stanford Open Policing Project,Standardized stop data from the Stanford Open Policing Project,https://openpolicing.stanford.edu/data/,https://github.com/stanford-policylab/opp/blob/master/data_readme.md,https://stacks.stanford.edu/file/druid:wb225bk3255/wb225bk3255_wi_madison_2023_01_26.csv.zip,CSV,date,NaN,<NA>,<NA>,<NA>,NaN
1327,Wisconsin,Milwaukee,Milwaukee,Milwaukee Police Department,COMPLAINTS,2019-01-01,202

In [ ]:
import datetime
import requests

In [ ]:
# Load the source file containing the dataset URLs
df = filtered_records.head(1)

In [ ]:
df

,State,SourceName,Agency,AgencyFull,TableType,coverage_start,coverage_end,last_coverage_check,Year,agency_originated,supplying_entity,Description,source_url,readme,URL,DataType,date_field,dataset_id,agency_field,min_version,py_min_version,query
73,California,California,MULTIPLE,NaN,USE OF FORCE - INCIDENTS,2023-01-01,2023-12-31,09/04/2024,2023,yes,California Office of the Attorney General,The Use of Force Incident Reporting incidents that result in serious bodily injury or death or involved the discharge of a firearm are reported annually from LEAs and other entities throughout the state that employ peace officers. The Use of Force Incident Reporting data is narrowly defined and does not represent the totality of use of force incidents that occur in California.,https://openjustice.doj.ca.gov/data,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-07/use-of-force-readme-06202024f.pdf,https://data-openjustice.doj.ca.gov/sites/default/files/dataset/2024-07/UseofForce_Incident_2023.csv,CSV,INCIDENT_DATE_STR,NaN,ORI,<NA>,<NA>,NaN


In [ ]:
# Define the threshold for outdated sources
def is_outdated(last_checked, threshold_days=1):
    """Check if the last checked date is older than the threshold."""
    last_checked_date = pd.to_datetime(last_checked)
    return (datetime.datetime.now() - last_checked_date).days > threshold_days

# Iterate through the DataFrame to process outdated sources
for index, row in df.iterrows():
    last_checked = row.get("Last Checked Date")
    if pd.isnull(last_checked) or is_outdated(last_checked):
        print(f"Processing outdated source: {row['source_url']}")

        # Fetch max coverage end year
        coverage_end = row.get("coverage_end")
        if pd.isnull(coverage_end):
            print(f"Skipping source {row['source_url']} due to missing coverage_end.")
            continue

        # Extract year from the URL
        source_url = row["source_url"]
        year_in_url = None
        for part in source_url.split("/"):
            if part.isdigit() and len(part) == 4:  # Check for a 4-digit year
                year_in_url = int(part)
                break

        if year_in_url is None:
            print(f"No year found in URL: {source_url}")
            continue

        # Replace year in URL with coverage_end year + 1
        new_year = int(coverage_end) + 1
        updated_url = source_url.replace(str(year_in_url), str(new_year))
        print(f"Updated URL: {updated_url}")
        # Try to open the new URL
        try:
            response = requests.get(updated_url)
            if response.status_code == 200:
                print(f"Data found at {updated_url}")
                df.at[index, "Last Checked Date"] = datetime.datetime.now().strftime("%Y-%m-%d")
            
                # Add a new row by copying the current row and updating the related data columns
                new_row = row.copy()
                new_row["coverage_start"] = pd.to_datetime(row["coverage_end"]) + pd.Timedelta(days=1)
                new_row["coverage_end"] = pd.to_datetime(row["coverage_end"]) + pd.DateOffset(years=1)
                new_row["source_url"] = updated_url
                new_row["last_coverage_check"] = datetime.datetime.now().strftime("%m/%d/%Y")
                new_df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
            else:
                print(f"Failed to fetch data from {updated_url}. Status code: {response.status_code}")
        except Exception as e:
            print(f"Error fetching data from {updated_url}: {e}")

Processing outdated source: https://www.beloitwi.gov/index.asp?SEC=3209CC74-34C0-4358-A679-9B061EE965C3&DE=FA507A9C-2ED5-4E8C-B098-8F3E78568AF6
No year found in URL: https://www.beloitwi.gov/index.asp?SEC=3209CC74-34C0-4358-A679-9B061EE965C3&DE=FA507A9C-2ED5-4E8C-B098-8F3E78568AF6
Processing outdated source: https://openpolicing.stanford.edu/data/
No year found in URL: https://openpolicing.stanford.edu/data/
Processing outdated source: https://data.milwaukee.gov/dataset/fpc-citizen-complaints
No year found in URL: https://data.milwaukee.gov/dataset/fpc-citizen-complaints
Processing outdated source: https://openpolicing.stanford.edu/data/
No year found in URL: https://openpolicing.stanford.edu/data/
Processing outdated source: https://openpolicing.stanford.edu/data/
No year found in URL: https://openpolicing.stanford.edu/data/
